#### Streaming the response

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("What did the author do growing up?")
response.print_response_stream()

The author focused on writing short stories and programming, particularly experimenting with early programming languages like Fortran on the IBM 1401 in 9th grade. Later, the author transitioned to working with microcomputers, building simple games and a word processor on a TRS-80 in the early 1980s.

#### a chatbot instead of Q&A

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_chat_engine()
response = query_engine.chat("What did the author do growing up?")
print(response)

response = query_engine.chat("Oh interesting, tell me more.")
print(response)

The author focused on writing and programming before college.
The author focused on writing short stories and programming before college. Their early stories lacked plot but focused on characters with strong emotions. In terms of programming, they started with an IBM 1401 using an early version of Fortran, where they struggled due to the limited input options available. The author's interest in programming grew with the introduction of microcomputers, particularly when they acquired a TRS-80 and began writing simple games and programs, including a word processor. Despite enjoying programming, the author initially planned to study philosophy in college but eventually switched to AI due to finding philosophy courses uninteresting.


#### Using LLM'S

In [5]:
from llama_index.llms.openai import OpenAI

response = OpenAI().complete("Paul Graham is ")
print(response)

a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on programming languages and web development. Graham is also a prolific writer and has published essays on a wide range of topics, including startups, technology, and education.


In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("What did the author do growing up?")
response.print_response_stream()

The author focused on writing short stories and programming, particularly experimenting with early programming languages like Fortran on an IBM 1401 in 9th grade. Later, with the introduction of microcomputers, the author's interest in programming grew as they started writing simple games and other programs on a TRS-80 computer.

#### Using a local LLM


In [7]:
# from llama_index.llms.ollama import Ollama
# from llama_index.core import Settings

# Settings.llm = Ollama(model="llama2", request_timeout=60.0)

#### Providing your own prompt template

In [10]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=..., query_str=...)

# or easily convert to message prompts (for chat API)
messages = qa_template.format_messages(context_str=..., query_str=...)

### Node Parser Usage Pattern

Node parsers are a simple abstraction that take a list of documents, and chunk them into Node objects, such that each node is a specific chunk of the parent document. When a document is broken into nodes, all of it’s attributes are inherited to the children nodes (i.e. metadata, text and metadata templates, etc.). You can read more about Node and Document properties.

#### Standalone Usage
Node parsers can be used on their own

In [11]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

nodes = node_parser.get_nodes_from_documents(
    [Document(text="long text")], show_progress=False
)

#### Index Usage
Or set inside a transformations or global settings to be used automatically when an index is constructed using .from_documents():

In [14]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("./data").load_data()

# global
from llama_index.core import Settings

Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

# per-index
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)],
)

In [16]:
import asyncio
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

async def main():
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(chunk_size=25, chunk_overlap=0),
            TitleExtractor(),
            OpenAIEmbedding(),
        ]
    )

    # run the pipeline
    nodes = await pipeline.run_async(documents=[Document.example()])

# Run the asynchronous main function
asyncio.create_task(main())


<Task pending name='Task-4' coro=<main() running at /var/folders/yq/ssn56htj0_jdf574hksbbxw00000gn/T/ipykernel_74097/2431013053.py:8>>